In [1]:
from soundgraph.graph import SoundGraph, SoundNode
from soundgraph.utils import start_session
from soundgraph.nodes import AudioReader, Mixer, Sampler, Oscillator, Scaler, MidiInput, MidiMapper
import librosa
import numpy as np

session = start_session()

samples = dict(
    kick = '../samples/drumkits/Roland TR-808/Bassdrum-01.mp3',
    hh_closed = '../samples/drumkits/Roland TR-808/Hat Closed.mp3',
    hh_open = '../samples/drumkits/Roland TR-808/Hat Open.mp3',
    snare = '../samples/drumkits/Roland TR-808/Snaredrum.mp3',
    crash = '../samples/drumkits/Roland TR-808/Crash-01.mp3'
)

beat = dict(
    kick =      [1,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0],
    hh_closed = [1,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0],
    hh_open =   [0,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0],
    crash =     [1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    snare =     [0,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0]
)

sampler = Sampler(name='sampler')
for k,v in samples.items():
    sampler.update_sample(k,v)
    
sampler.update('beat',beat)

sound_graph = SoundGraph()
sound_graph.add(sampler)
sound_graph.set_output('sampler.out')
session.set_graph(sound_graph)

Starting new session
Use default output devices? [(Y)/n)]
Use default input devices? [(Y)/n)]


/home/lpepino/anaconda3/lib/python3.7/site-packages/librosa/core/audio.py:162: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/home/lpepino/anaconda3/lib/python3.7/site-packages/librosa/core/audio.py:162: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/home/lpepino/anaconda3/lib/python3.7/site-packages/librosa/core/audio.py:162: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/home/lpepino/anaconda3/lib/python3.7/site-packages/librosa/core/audio.py:162: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/home/lpepino/anaconda3/lib/python3.7/site-packages/librosa/core/audio.py:162: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audior

In [2]:
sampler.update('bpm',100)
sampler.update('divisions_per_bar',4)
sampler.update('bars',4)

beat = dict(
    kick =      [1,0,0,1,0,1,0,1,0,1,0,0,1,1,1,1],
    hh_closed = [1,0,1,0,0,0,1,0,1,1,0,0,1,0,1,0],
    hh_open =   [0,0,0,0,1,1,0,1,0,0,1,0,1,0,0,0],
    crash =     [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    snare =     [1,0,0,0,1,0,0,0,0,0,1,0,1,1,0,0]
)

sampler.update('beat',beat)

In [3]:
osc1 = Oscillator(name='osc1')
sound_graph.add(osc1)
mixer = Mixer(name='mixer')
sound_graph.add(mixer)
sound_graph.connect('osc1.out','mixer.in_1')
sound_graph.connect('sampler.out','mixer.in_2')
mixer.update('volume_1',0.3)
mixer.update('volume_2',0.7)
sound_graph.set_output('mixer.out')

In [4]:
lfo1 = Oscillator(name='lfo1')
sound_graph.add(lfo1)
sound_graph.connect('lfo1.out','osc1.frequency')

In [5]:
lfo1.update('frequency',12)
lfo1.update('amplitude',30)
lfo1.update('offset',550)
lfo1.update('osc_type','square')
osc1.update('osc_type','square')
mixer.update('volume_1',0.08)
mixer.update('volume_2',0.9)

In [6]:
lx61_controls = MidiInput(name='lx61_controls')
lx61_controls.update('port','Impact LX61+:Impact LX61+ MIDI 2 32:1')
freq_mapping = MidiMapper(name='freq_mapper')
freq_mapping_config = dict(
    channel=15,
    control=63,
    map_parameter='value',
    mapping=np.logspace(1,np.log(8000)/np.log(20),128,base=20)
)
freq_mapping.update('mapping',freq_mapping_config)
sound_graph.add(freq_mapping)
sound_graph.add(lx61_controls)
sound_graph.connect('lx61_controls.out','freq_mapper.midi_in')
sound_graph.connect('freq_mapper.out','lfo1.offset')

In [7]:
lfo_mapping = MidiMapper(name='lfo_mapper')
lfo_mapping_config = dict(
    channel=15,
    control=62,
    map_parameter='value',
    mapping=np.linspace(0.01,50,128)
)
lfo_mapping.update('mapping',lfo_mapping_config)
sound_graph.add(lfo_mapping)
sound_graph.connect('lx61_controls.out','lfo_mapper.midi_in')
sound_graph.connect('lfo_mapper.out','lfo1.frequency')

In [25]:
scaler_freq = Scaler(name='scale_sampler_freq')
scaler_amp = Scaler(name='scale_sampler_amp')
scaler_freq.update('scale',1000)
scaler_amp.update('scale',100)
sound_graph.add(scaler_freq)
sound_graph.add(scaler_amp)
sound_graph.connect('sampler.out','scale_sampler_freq.input')
sound_graph.connect('sampler.out','scale_sampler_amp.input')
sound_graph.connect('scale_sampler_freq.out','lfo1.offset')
sound_graph.connect('scale_sampler_amp.out','lfo1.amplitude')

In [33]:
scaler_freq.update('scale',3000)
scaler_amp.update('scale',2000)
lfo1.update('frequency',0.05)
lfo1.update('osc_type','sin')

In [8]:
lfo1.update('osc_type','sin')